In [ ]:
import os
import openai
import dotenv
import pyodbc 
import json

dotenv.load_dotenv()

In [ ]:
use_azure_active_directory = False  # Set this flag to True if you are using Azure Active Directory

In [ ]:
if not use_azure_active_directory:
    endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
    api_key = os.getenv("AZURE_OPENAI_API_KEY")

    client = openai.AzureOpenAI(
        azure_endpoint=endpoint,
        api_key=api_key,
        api_version="2024-02-15-preview"
    )

In [ ]:
deployment = "gpt-4" # Fill in the deployment name from the portal here

In [ ]:
functions = [
        {
            "type": "function",
            "function": {
                "name": "get_customers",
                "description": "Get the customers for a given city",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "city": {
                            "type": "string",
                            "description": "The city, e.g. New York",
                        }
                    },
                    "required": ["city"],
                },
            },
        }
]

In [ ]:
messages = [
    {"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."},
    {"role": "user", "content": "Show me customers list for New York city."}
]

chat_completion = client.chat.completions.create(
    model=deployment,
    messages=messages,
    tools=functions
)
print(chat_completion)





In [ ]:

def get_customers(request):
    """
    This function is for illustrative purposes.
    The city should be used to determine customer
    instead of returning a hardcoded response.
    """
    city = request.get("city")

    cnxn = pyodbc.connect('Driver={ODBC Driver 18 for SQL Server};Server=tcp:bikestore.database.windows.net,1433;Database=bikestore;Uid=' + os.getenv("SQL_DB_USER") + ';Pwd=' + os.getenv("SQL_DB_PASSWORD") + ';Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30;')
    cursor = cnxn.cursor()	

    sql = 'call SelectAllCustomersParams(?)'
    values = (city)

    sql = """\
    SET NOCOUNT ON;
    DECLARE @RC int;
    EXEC @RC = [bikestore].[dbo].[SelectAllCustomersParams] ?;
    SELECT @RC AS rc;
    """

    cursor.execute(sql, (values))

    rows = []
    row = cursor.fetchone() 
    while row:
        row = cursor.fetchone()
        rows.append(row)

    print(str(rows))
    return str(rows)

function_call = chat_completion.choices[0].message.tool_calls[0].function
print(function_call.name)
print(function_call.arguments)

if function_call.name == "get_customers":
    response = get_customers(json.loads(function_call.arguments))
    print(response)

In [ ]:
"""
messages.append(
    {
        "role": "function",
        "name": "get_customers",
        "content": json.dumps(response)
    }
)

function_completion = client.chat.completions.create(
    model=deployment,
    messages=messages,
    tools=functions,
)

print(function_completion.choices[0].message.content.strip())
"""